In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import time
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### Вспомогательные функции

In [2]:
def create_cf_matrix(train, cf_type='user'):

    """
    Создает матрицу для коллаборативной фильтрации.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - Матрица для коллаборативной фильтрации.
    """
    if cf_type == 'user':
        # Матрица пользователь-фильм
        return train.pivot(index='userId', columns='movieId', values='rating')
    elif cf_type == 'item':
        # Матрица фильм-пользователь
        return train.pivot(index='movieId', columns='userId', values='rating')
    else:
        raise ValueError("cf_type должен быть 'user' или 'item'")
    
# Функция для замера времени
def time_function(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed_time = time.time() - start_time
    return result, elapsed_time

# Оформление результата
def create_results_dataframe(results):
    """
    Создает pandas DataFrame из словаря результатов.
    
    :param results: Словарь с названиями методов в качестве ключей и RMSE в качестве значений
    :return: pandas DataFrame с результатами
    """
    df = pd.DataFrame(results).T
    df.columns = ['RMSE', 'Время (с)']
    df.index.name = 'Метод'
    return df

# Загрузка и разделение данных
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    df = pd.read_csv(file_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test

### Основная часть

In [4]:
# 2-3. User-based and Item-based коллаборативная фильтрация с/без z-score
def collaborative_filtering(train, test, cf_type='user', use_mean_adjustment=False, use_z_score=False, N=None):
    # Используем отдельную функцию для создания матрицы
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    # Вычисляем косинусное сходство
    similarity = cosine_similarity(matrix.fillna(0))
    
    # Вычисляем средние и стандартные отклонения для Z-score (если нужно)
    means = matrix.mean(axis=1)
    stds = matrix.fillna(0).std(axis=1) if use_z_score else None

    # Если включен Z-score, нормализуем рейтинги
    if use_z_score:
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0).fillna(0)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            # Если тип фильтрации пользователь-фильм
            user_index, item_index = user_id, item_id
        else:
            # Если тип фильтрации фильм-пользователь
            user_index, item_index = item_id, user_id
        
        # Если пользователь или элемент отсутствует в базе
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()
        
        # Рейтинги текущего пользователя или фильма
        ratings = matrix.loc[user_index]
        # Сходства с другими пользователями или фильмами
        similarities = similarity[matrix.index.get_loc(user_index)]
        
        # Сортируем по схожести и выбираем топ-N (если задано)
        sorted_similarities = np.argsort(similarities)[::-1]
        top_similar = sorted_similarities[:N] if N else sorted_similarities
        
        numerator = 0
        denominator = 0
        for similar in top_similar:
            # Проверяем, есть ли рейтинг для указанного элемента (пользователя или фильма)
            if matrix.iloc[similar][item_index] > 0:
                if use_z_score:
                    # Коррекция на Z-score
                    adjusted_rating = matrix_normalized.iloc[similar][item_index]
                    numerator += similarities[similar] * adjusted_rating
                elif use_mean_adjustment:
                    # Коррекция на среднюю оценку
                    adjusted_rating = matrix.iloc[similar][item_index] - means.iloc[similar]
                    numerator += similarities[similar] * adjusted_rating
                else:
                    numerator += similarities[similar] * matrix.iloc[similar][item_index]
                
                # Знаменатель: \text{denominator} += \text{sim}(u, v)
                denominator += abs(similarities[similar])
        
        # Если нет похожих пользователей/фильмов, предсказываем средний рейтинг пользователя
        if denominator == 0:
            return ratings.mean()
        
        if use_z_score:
            # Предсказание через Z-score
            predicted_z_score = numerator / denominator
            return means.loc[user_index] + predicted_z_score * stds.loc[user_index]
        elif use_mean_adjustment:
            # Предсказание с коррекцией на среднее значение
            return means.loc[user_index] + numerator / denominator
        else:
            # Предсказание без коррекции на среднее значение
            return numerator / denominator
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

In [5]:
# 4-5. Рекомендательная система на основе кластеризации и расчета средней оценки кластера
def cluster_based_cf(train, test, n_clusters=40, use_z_score=False, cf_type='user'):
    """
    Кластерная коллаборативная фильтрация.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - test: DataFrame с тестовыми данными.
    - n_clusters: количество кластеров для K-means.
    - use_z_score: флаг для использования Z-score нормализации.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - RMSE для предсказанных значений.
    """
    # Создаем матрицу с помощью create_cf_matrix
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    if use_z_score:
        # Нормализуем оценки с помощью z-score
        means = matrix.mean(axis=1)
        stds = matrix.fillna(0).std(axis=1)
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0)
        matrix_for_clustering = matrix_normalized.fillna(0)
    else:
        # Используем обычную матрицу для кластеризации
        matrix_for_clustering = matrix.fillna(0)
    
    # Применяем K-means кластеризацию
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(matrix_for_clustering)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id
        
        # Проверка, если пользователя или элемента нет в матрице
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()
        
        # Определяем кластер пользователя или фильма
        cluster = clusters[matrix.index.get_loc(user_index)]
        cluster_items = matrix.index[clusters == cluster]
        
        if use_z_score:
            # Получаем нормализованные рейтинги для кластера
            cluster_ratings_normalized = matrix_normalized.loc[cluster_items, item_index]
            
            # Проверяем, пусты ли нормализованные рейтинги
            if cluster_ratings_normalized.empty or cluster_ratings_normalized.isna().all():
                return matrix.mean().mean()  # Если нет рейтингов, вернуть среднее по всей матрице
            
            predicted_z_score = cluster_ratings_normalized.mean()
            # Преобразуем предсказание обратно в исходные единицы
            return means[user_index] + (predicted_z_score * stds[user_index])
        else:
            # Получаем обычные рейтинги для кластера
            cluster_ratings = matrix.loc[cluster_items, item_index]
            
            # Проверка на наличие не NaN значений
            if cluster_ratings.empty or cluster_ratings.isna().all():
                return matrix.mean().mean()  # Если нет доступных значений, вернуть среднее по всей матрице
            
            # Возвращаем среднее значение по кластеру
            return cluster_ratings.mean()
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

In [6]:
# 6. Рекомендательная система на основе голосования
def voting_based_cf(train, test, cf_type='user', n_neighbors=40, use_knn=False):
    """
    Коллаборативная фильтрация с голосованием на основе сходства и KNN.

    Параметры:
    - train: DataFrame с обучающими данными.
    - test: DataFrame с тестовыми данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    - n_neighbors: количество соседей для KNN или топ-N для голосования.
    - use_knn: флаг для использования KNN вместо простого выбора топ-N похожих пользователей/фильмов.

    Возвращает:
    - RMSE для предсказанных значений.
    """
    # Создаем матрицу для коллаборативной фильтрации
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    # Если используем KNN
    if use_knn:
        knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(matrix.fillna(0))
    
    # Вычисляем косинусное сходство, если не используем KNN
    similarity = cosine_similarity(matrix.fillna(0)) if not use_knn else None

    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id

        # Если пользователя или элемента нет в матрице
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()

        # Если используем KNN
        if use_knn:
            # Найти ближайших соседей для пользователя
            distances, neighbors = knn.kneighbors([matrix.fillna(0).loc[user_index]], n_neighbors=n_neighbors)
            neighbors = neighbors.flatten()  # Индексы соседей
        else:
            # Вычисляем схожесть и выбираем топ-N похожих пользователей/фильмов
            similarities = similarity[matrix.index.get_loc(user_index)]
            neighbors = np.argsort(similarities)[-n_neighbors-1:-1]  # Топ-N соседей

        # Собираем рейтинги соседей для указанного элемента
        neighbor_ratings = matrix.iloc[neighbors][item_index].values
        valid_ratings = neighbor_ratings[neighbor_ratings > 0]

        # Если нет валидных рейтингов, возвращаем средний рейтинг пользователя
        if len(valid_ratings) == 0:
            return matrix.loc[user_index].mean()

        # Возвращаем среднее значение рейтингов соседей
        return valid_ratings.mean()

    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем RMSE
    rmse = np.sqrt(mean_squared_error(test['rating'], test['predicted_rating']))
    return rmse

In [60]:
# Основная функция
def main():
    # Загрузка и разделение данных
    train, test = load_and_split_data('data/ml-latest-small/ratings.csv', test_size=0.05)
    
    results = {}

    # User-based CF
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='user', N=100)
    results['user_based'] = [rmse, elapsed_time]
    logger.info(f'user_based ✔ (время: {elapsed_time:.2f} с)')

    # User-based CF with Mean
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='user', use_mean_adjustment=True, N=100)
    results['user_based_mean'] = [rmse, elapsed_time]
    logger.info(f'user_based_mean ✔ (время: {elapsed_time:.2f} с)')

    # User-based CF with z-score
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='user', use_z_score=True, N=100)
    results['user_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'user_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Item-based CF
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='item', N=100)
    results['item_based'] = [rmse, elapsed_time]
    logger.info(f'item_based ✔ (время: {elapsed_time:.2f} с)')

    # Item-based CF with Mean
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='item', use_mean_adjustment=True, N=100)
    results['item_based_mean'] = [rmse, elapsed_time]
    logger.info(f'item_based_mean ✔ (время: {elapsed_time:.2f} с)')

    # Item-based CF with z-score
    (rmse, elapsed_time) = time_function(collaborative_filtering, train, test, cf_type='item', use_z_score=True, N=100)
    results['item_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'item_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=False, cf_type='user')
    results['user_cluster_based'] = [rmse, elapsed_time]
    logger.info(f'user_cluster_based ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF with z-score
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=True, cf_type='user')
    results['user_cluster_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'user_cluster_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=False, cf_type='item')
    results['item_cluster_based'] = [rmse, elapsed_time]
    logger.info(f'item_cluster_based ✔ (время: {elapsed_time:.2f} с)')

    # Cluster-based CF with z-score
    (rmse, elapsed_time) = time_function(cluster_based_cf, train, test, n_clusters=5, use_z_score=True, cf_type='item')
    results['item_cluster_based_z-score'] = [rmse, elapsed_time]
    logger.info(f'item_cluster_based_z-score ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='user', n_neighbors=40, use_knn=False)
    results['user_voting_based'] = [rmse, elapsed_time]
    logger.info(f'user_voting_based ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='user', n_neighbors=40, use_knn=True)
    results['user_voting_based_knn'] = [rmse, elapsed_time]
    logger.info(f'user_voting_based_knn ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='item', n_neighbors=40, use_knn=False)
    results['item_voting_based'] = [rmse, elapsed_time]
    logger.info(f'item_voting_based ✔ (время: {elapsed_time:.2f} с)')

    # Voting-based CF
    (rmse, elapsed_time) = time_function(voting_based_cf, train, test, cf_type='item', n_neighbors=40, use_knn=True)
    results['item_voting_based_knn'] = [rmse, elapsed_time]
    logger.info(f'item_voting_based_knn ✔ (время: {elapsed_time:.2f} с)')

    # Создаем DataFrame с результатами
    results_df = create_results_dataframe(results)
    
    # Выводим результаты
    logger.info(f'\n{results_df.sort_values("RMSE")}')
    return results_df

results_df = main()

INFO:__main__:user_based ✔ (время: 33.66 с)
INFO:__main__:user_based_mean ✔ (время: 34.62 с)
INFO:__main__:user_based_z-score ✔ (время: 34.16 с)
INFO:__main__:item_based ✔ (время: 50.52 с)
INFO:__main__:item_based_mean ✔ (время: 51.31 с)
INFO:__main__:item_based_z-score ✔ (время: 50.23 с)
INFO:__main__:user_cluster_based ✔ (время: 48.52 с)
INFO:__main__:user_cluster_based_z-score ✔ (время: 19.56 с)
INFO:__main__:item_cluster_based ✔ (время: 31.40 с)
INFO:__main__:item_cluster_based_z-score ✔ (время: 33.44 с)
INFO:__main__:user_voting_based ✔ (время: 21.38 с)
INFO:__main__:user_voting_based_knn ✔ (время: 458.48 с)
INFO:__main__:item_voting_based ✔ (время: 31.52 с)
INFO:__main__:item_voting_based_knn ✔ (время: 467.31 с)
INFO:__main__:
                                RMSE   Время (с)
Метод                                           
item_based_z-score          0.894107   50.225999
item_based_mean             0.897859   51.305034
user_based_mean             0.917934   34.621960
item_based  

In [61]:
train, test = load_and_split_data('data/ml-latest-small/ratings.csv', test_size=0.05)

user_cf_matrix = create_cf_matrix(train, cf_type='user')
item_cf_matrix = create_cf_matrix(train, cf_type='item')

def average_interactions_per_user(matrix):
    interactions_per_user = matrix.count(axis=1)
    
    mean_value = interactions_per_user.mean()
    mode_value = interactions_per_user.mode()[0]
    median_value = interactions_per_user.median()
    
    return mean_value, mode_value, median_value

mean, mode, median = average_interactions_per_user(user_cf_matrix)
print(f"User-based матрица:\nСреднее: {mean:.4}, Мода: {mode}, Медиана: {median}")
mean, mode, median = average_interactions_per_user(item_cf_matrix)
print(f"Item-based матрица:\nСреднее: {mean:.4}, Мода: {mode}, Медиана: {median}")


User-based матрица:
Среднее: 157.0, Мода: 20, Медиана: 67.0
Item-based матрица:
Среднее: 10.02, Мода: 1, Медиана: 3.0


### Выводы

Сравнить качество реализованных подходов к рекомендациям и сделать вывод об их применимости.

Для набора данных при разделении на 5% тестовых данных (время работы алгоритмов довольно быстро увеличивается, а результаты похожи) лучше всего сработал item-based метод, однако при изменении размера выборки лучшим может оказаться и user-based. В целом значения RMSE всех алгоритмов находятся в диапазоне [0.89, 1.12]. Я полагал, что из-за большего количества данных для матрицы пользователь-фильм результаты user-based будут лучше, однако это не всегда так. Пример по количеству оценок в таблице:

| Матрица             | Среднее | Мода | Медиана |
|---------------------|---------|------|---------|
| **User-based**       | 157   | 20   | 67    |
| **Item-based**       | 10.02   | 1    | 3     |

Видно, что для **item-based** значения гораздо меньше, то есть фильмы, чаще всего, имеют лишь несколько оценок.

* Среди item-based лучше работает z-score. Однако, использование среднего для оценки также хорошо себя показало.

* Среди user-based лучше сработал вариант со средним, но использование z-score, в целом, не хуже.

* Использование KNN ухудшило метрики, а в некоторых алгоритмах увеличило время работы в 10 раз. Возможно, из-за плохой реализации.